In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import neurobiases
import numpy as np
import os

from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from sklearn.utils.extmath import cartesian

%matplotlib inline

In [ ]:
results_path = os.path.join(os.environ['HOME'], 'fits/neurobiases/exp4.npz')

In [ ]:
results = np.load(results_path)

In [ ]:
list(results)

In [ ]:
a_true = results['a_true'][:]
a_est = results['a_est'][:]
b_true = results['b_true'][:]
b_est = results['b_est'][:]
B_true = results['B_true'][:].reshape((10, 15))
B_est = results['B_est'][:]
Psi_true = results['Psi_true'][:]
Psi_est = results['Psi_est'][:]
scores = results['scores'][:].reshape((30, 30, -1))
bics = results['bics'][:].reshape((30, 30, -1))
n_iterations = results['n_iterations'][:]

In [ ]:
N = a_true.size
M = b_true.size

In [ ]:
ticks = [0, 6, 12, 18, 24, 29]
xticklabels = ['1e-6', '1e-5', '1e-4', '1e-3', '1e-2', '1e-1']
yticklabels = ['1e-4', '1e-3', '1e-2', '1e-1', '1e-0', '1e1']

# Selection Ratio

In [ ]:
a_srs = np.median((np.count_nonzero(a_est, axis=2) / N).reshape((30, 30, -1)), axis=2)
b_srs = np.median((np.count_nonzero(b_est, axis=2) / M).reshape((30, 30, -1)), axis=2)
B_srs = np.median((np.count_nonzero(B_est, axis=(2, 3)) / (N * M)).reshape((30, 30, -1)), axis=2)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 4), sharex=True, sharey=True)

axes[0].imshow(a_srs, vmin=0, vmax=1)
axes[1].imshow(b_srs, vmin=0, vmax=1)
img = axes[2].imshow(B_srs, vmin=0, vmax=1)

axes[0].set_xlabel(r'\textbf{Tuning Penalty}', fontsize=16)
axes[0].set_ylabel(r'\textbf{Coupling Penalty}', fontsize=16)

axes[0].set_xticks(ticks)
axes[1].set_yticks(ticks)

for ax in axes:
    ax.set_xticklabels(xticklabels, fontsize=12)
    ax.set_yticklabels(yticklabels, fontsize=12)
    
axes[0].set_title(r'\textbf{Coupling}', fontsize=18)
axes[1].set_title(r'\textbf{Target Tuning}', fontsize=18)
axes[2].set_title(r'\textbf{Non-target Tuning}', fontsize=18)

[[x0, y0], [x1, y1]] = axes[2].get_position().get_points()
cax = fig.add_axes([x1 + 0.05 * (x1 - x0), y0, 0.05 * (x1 - x0), (y1 - y0)])
cb = fig.colorbar(img, cax=cax)
cb.ax.tick_params(labelsize=10)
cb.ax.set_ylabel(r'\textbf{Selection Ratio}', fontsize=15, labelpad=15, rotation=270)

plt.show()

# Selection Accuracy

In [ ]:
a_est_masks = a_est != 0
b_est_masks = b_est != 0
B_est_masks = B_est != 0

In [ ]:
b_mask = b_true != 0
a_mask = a_true != 0
B_mask = B_true != 0

In [ ]:
a_norm = np.sum(a_est_masks, axis=2) + np.sum(a_mask)
b_norm = np.sum(b_est_masks, axis=2) + np.sum(b_mask)
B_norm = np.sum(B_est_masks, axis=(2, 3)) + np.sum(B_mask)

In [ ]:
a_sa = np.median(1 - np.sum(a_est_masks != a_mask, axis=2) / a_norm, axis=1).reshape((30, 30))
b_sa = np.median(1 - np.sum(b_est_masks != b_mask, axis=2) / b_norm, axis=1).reshape((30, 30))
B_sa = np.median(1 - np.sum(B_est_masks != B_mask, axis=(2, 3)) / B_norm, axis=1).reshape((30, 30))

In [ ]:
mean_sa = (a_sa + b_sa + B_sa) / 3

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(14, 4), sharex=True, sharey=True)

axes[0].imshow(a_sa, vmin=0, vmax=1)
axes[1].imshow(b_sa, vmin=0, vmax=1)
axes[2].imshow(B_sa, vmin=0, vmax=1)
img = axes[3].imshow(mean_sa, vmin=0, vmax=1)

axes[0].set_xlabel(r'\textbf{Tuning Penalty}', fontsize=16)
axes[0].set_ylabel(r'\textbf{Coupling Penalty}', fontsize=16)

axes[0].set_xticks(ticks)
axes[1].set_yticks(ticks)

for ax in axes:
    ax.set_xticklabels(xticklabels, fontsize=12)
    ax.set_yticklabels(yticklabels, fontsize=12)
    
axes[0].set_title(r'\textbf{Coupling}', fontsize=18)
axes[1].set_title(r'\textbf{Target Tuning}', fontsize=18)
axes[2].set_title(r'\textbf{Non-target Tuning}', fontsize=18)
axes[3].set_title(r'\textbf{Overall}', fontsize=18)

[[x0, y0], [x1, y1]] = axes[3].get_position().get_points()
cax = fig.add_axes([x1 + 0.05 * (x1 - x0), y0, 0.05 * (x1 - x0), (y1 - y0)])
cb = fig.colorbar(img, cax=cax)
cb.ax.tick_params(labelsize=10)
cb.ax.set_ylabel(r'\textbf{Selection Accuracy}', fontsize=15, labelpad=15, rotation=270)

plt.show()

# Cross-validated Log-likelihood

In [ ]:
mean_lls = np.mean(scores, axis=2)
best_ll = np.unravel_index(np.argmax(mean_lls), mean_lls.shape)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(np.sort(-mean_lls.ravel()), linewidth=3, color='black')
ax.set_xlabel(r'\textbf{CV Index}', fontsize=20)
ax.set_ylabel(r'\textbf{Mean CV Negative LL}', fontsize=20)

ax.set_xlim([0, 900])
ax.set_ylim([-np.max(mean_lls), -np.percentile(mean_lls, q=20)])
ax.tick_params(labelsize=15)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

img = ax.imshow(mean_lls, vmin=np.percentile(mean_lls, q=90), vmax=np.max(mean_lls))
ax.set_xlabel(r'\textbf{Tuning Penalty}', fontsize=16)
ax.set_ylabel(r'\textbf{Coupling Penalty}', fontsize=16)

ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(xticklabels, fontsize=12)
ax.set_yticklabels(yticklabels, fontsize=12)

ax.scatter(best_ll[1], best_ll[0], color='red', marker='x')

[[x0, y0], [x1, y1]] = ax.get_position().get_points()
cax = fig.add_axes([x1 + 0.05 * (x1 - x0), y0, 0.05 * (x1 - x0), (y1 - y0)])
cb = fig.colorbar(img, cax=cax)
cb.ax.tick_params(labelsize=10)
cb.ax.set_ylabel(r'\textbf{Held out Log-Likelihood}', fontsize=15, labelpad=20, rotation=270)
plt.show()

# Bayesian Information Criterion

In [ ]:
mean_bics = np.mean(bics, axis=-1)
best_bic = np.unravel_index(np.argmin(mean_bics), mean_bics.shape)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(np.sort(mean_bics.ravel()), linewidth=3, color='black')
ax.set_xlabel(r'\textbf{CV Index}', fontsize=20)
ax.set_ylabel(r'\textbf{Mean BIC}', fontsize=20)

ax.set_xlim([0, 900])
ax.set_ylim([mean_bics.min(), mean_bics.max()])
ax.tick_params(labelsize=15)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

img = ax.imshow(mean_bics, vmin=np.min(mean_bics), vmax=np.percentile(mean_bics, q=20))

ax.set_xlabel(r'\textbf{Tuning Penalty}', fontsize=16)
ax.set_ylabel(r'\textbf{Coupling Penalty}', fontsize=16)

ax.set_xticks([0, 7, 15, 22, 29])
ax.set_yticks([0, 7, 15, 22, 29])
ax.set_xticklabels(['1e-5', '1e-4', '1e-3', '1e-2', '1e-1'], fontsize=12)
ax.set_yticklabels(['1e-4', '1e-3', '1e-2', '1e-1', '1e-0'], fontsize=12)

ax.scatter(best_bic[1], best_bic[0], color='red', marker='x')

[[x0, y0], [x1, y1]] = ax.get_position().get_points()
cax = fig.add_axes([x1 + 0.05 * (x1 - x0), y0, 0.05 * (x1 - x0), (y1 - y0)])
cb = fig.colorbar(img, cax=cax)
cb.ax.tick_params(labelsize=10)
cb.ax.set_ylabel(r'\textbf{BIC}', fontsize=15, labelpad=20, rotation=270)

plt.show()

# Bias 

In [ ]:
b_mask = b_true != 0
a_mask = a_true != 0
a_nz = a_true[a_mask]
b_nz = b_true[b_mask]

In [ ]:
a_bias = np.mean(np.median(a_est - a_true, axis=1).reshape((30, 30, -1)), axis=-1)
b_bias = np.mean(np.median(b_est - b_true, axis=1).reshape((30, 30, -1)), axis=-1)
B_bias = np.mean(np.median(B_est - B_true, axis=1).reshape((30, 30, 10, 15)), axis=(2, 3))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 6), sharex=True, sharey=True)

plt.subplots_adjust(wspace=0.5)

img1 = axes[0].imshow(a_bias, vmin=-0.2, vmax=0.2, cmap='RdGy')
img2 = axes[1].imshow(b_bias, vmin=-30, vmax=30, cmap='RdGy')
img3 = axes[2].imshow(B_bias, vmin=-30, vmax=30, cmap='RdGy')

axes[0].set_xticks(ticks)
axes[1].set_yticks(ticks)

for ax in axes:
    ax.set_xticklabels(xticklabels, fontsize=12)
    ax.set_yticklabels(yticklabels, fontsize=12)
    ax.scatter(best_ll[1], best_ll[0], color='purple', marker='x', s=50)
    ax.scatter(best_bic[1], best_bic[0], color='cyan', marker='x', s=50)

axes[0].set_title(r'\textbf{Coupling}', fontsize=18)
axes[1].set_title(r'\textbf{Target Tuning}', fontsize=18)
axes[2].set_title(r'\textbf{Non-target Tuning}', fontsize=18)

[[x0, y0], [x1, y1]] = axes[0].get_position().get_points()
cax1 = fig.add_axes([x1 + 0.05 * (x1 - x0), y0, 0.05 * (x1 - x0), (y1 - y0)])
cb = fig.colorbar(img1, cax=cax1)
cb.ax.tick_params(labelsize=10)
cb.ax.set_ylabel(r'\textbf{Bias}', fontsize=15, labelpad=20, rotation=270)

[[x0, y0], [x1, y1]] = axes[1].get_position().get_points()
cax1 = fig.add_axes([x1 + 0.05 * (x1 - x0), y0, 0.05 * (x1 - x0), (y1 - y0)])
cb = fig.colorbar(img2, cax=cax1)
cb.ax.tick_params(labelsize=10)
cb.ax.set_ylabel(r'\textbf{Bias}', fontsize=15, labelpad=20, rotation=270)

[[x0, y0], [x1, y1]] = axes[2].get_position().get_points()
cax1 = fig.add_axes([x1 + 0.05 * (x1 - x0), y0, 0.05 * (x1 - x0), (y1 - y0)])
cb = fig.colorbar(img3, cax=cax1)
cb.ax.tick_params(labelsize=10)
cb.ax.set_ylabel(r'\textbf{Bias}', fontsize=15, labelpad=20, rotation=270)

plt.show()

In [ ]:
a_nz_bias = np.mean(np.median((a_est[..., a_mask] - a_nz)/np.abs(a_nz), axis=1).reshape((30, 30, -1)), axis=-1)
b_nz_bias = np.mean(np.median((b_est[..., b_mask] - b_nz)/np.abs(b_nz), axis=1).reshape((30, 30, -1)), axis=-1)

In [ ]:
np.unravel_index(best_ll, (30, 30))

In [ ]:
### first plot
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True)

plt.subplots_adjust(wspace=0.5)

img1 = axes[0].imshow(a_nz_bias, vmin=-2, vmax=2, cmap='RdGy')
img2 = axes[1].imshow(b_nz_bias, vmin=-2, vmax=2, cmap='RdGy')

axes[0].set_xticks([0, 7, 15, 22, 29])
axes[1].set_yticks([0, 7, 15, 22, 29])

axes[0].set_xticks(ticks)
axes[1].set_yticks(ticks)

for ax in axes:
    ax.set_xticklabels(xticklabels, fontsize=12)
    ax.set_yticklabels(yticklabels, fontsize=12)
    
axes[0].set_title(r'\textbf{Coupling}', fontsize=18)
axes[1].set_title(r'\textbf{Target Tuning}', fontsize=18)

for ax in axes:
    ax.scatter(15, 15, color='blue', marker='x')
    ax.scatter(15, 25, color='purple', marker='x')
    ax.scatter(23, 13, color='green', marker='x')
    ax.scatter(20, 2, color='cyan', marker='x')


[[x0, y0], [x1, y1]] = axes[0].get_position().get_points()
cax1 = fig.add_axes([x1 + 0.05 * (x1 - x0), y0, 0.05 * (x1 - x0), (y1 - y0)])
cb = fig.colorbar(img1, cax=cax1)
cb.ax.tick_params(labelsize=10)
cb.ax.set_ylabel(r'\textbf{Normalized Bias}', fontsize=15, labelpad=20, rotation=270)

[[x0, y0], [x1, y1]] = axes[1].get_position().get_points()
cax1 = fig.add_axes([x1 + 0.05 * (x1 - x0), y0, 0.05 * (x1 - x0), (y1 - y0)])
cb = fig.colorbar(img2, cax=cax1)
cb.ax.tick_params(labelsize=10)
cb.ax.set_ylabel(r'\textbf{Normalized Bias}', fontsize=15, labelpad=20, rotation=270)
plt.show()

### second plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
colors = ['blue', 'purple', 'green', 'cyan']

axes[0].plot(a_true, color='k', marker='o', linewidth=2)
for idx, plot_idx in enumerate(plot_idxs):
    axes[0].plot(np.median(a_est[plot_idx], axis=0), color=colors[idx], marker='o', linewidth=3)
    
axes[1].plot(b_true, color='k', marker='o', linewidth=2)
for idx, plot_idx in enumerate(plot_idxs):
    axes[1].plot(np.median(b_est[plot_idx], axis=0), color=colors[idx], marker='o', linewidth=3)
    
for ax in axes:
    ax.tick_params(labelsize=10)
    ax.set_xticks([])
    
axes[0].set_ylim([-1.5, 1.5])
axes[1].set_ylim([-50, 175])

axes[0].set_ylabel(r'\textbf{Coupling Parameters}', fontsize=15)
axes[1].set_ylabel(r'\textbf{Tuning Parameters}', fontsize=15)
plt.show()